A test for qunantitative finance

Part 1:Getting Data

In [12]:
#获取数据
import numpy as np
import pandas as pd
import tushare as ts
import matplotlib.pyplot as plt
from datetime import datetime

pro = ts.pro_api('577f92d0103aa2c27d6c458f8b67817ccacd487492b50ce107d04fb0')

# #读取板块企业的上市日期和股票代码
# info_web3=pd.read_excel('C:\\Users\\86150\\Desktop\\大创\\under web 3.0 cooperations.xlsx')

# info_web3.describe()
# company_code=info_web3.iloc[:,0]
# listing_date=info_web3.iloc[:,2]

# data = {'A': [1, 2, 3, 4, 3], 'B': [5, 6, 7, 8, 9]}
# df = pd.DataFrame(data)

# #筛选出上市日期在2018年之前的企业
# before2018=info_web3[info_web3['上市日期']<=20171231]



# #获得企业数量
# company_num=before2018.shape[0]
# print(company_num)

# 选取000829天音控股进行学习与估计，选取其上市至20017年的数据进行训练，2018至2022年的数据进行测试
data_000829=pro.daily(ts_code='000829.sz',start_date='19971202',end_date='20221231')
data_000829=pd.DataFrame(data_000829)
data_000829 = data_000829.iloc[::-1].reset_index(drop=True)


In [13]:
#选取特定数据
data_000829=data_000829.loc[:,['trade_date','open','close','high','low']]

In [14]:
#转换日期格式

def weekday_to_chinese(weekday_number):
    # 中文星期几名称映射
    weekday_names = {
        0: "一",
        1: "二",
        2: "三",
        3: "四",
        4: "五",
        5: "六",
        6: "日"
    }
    
    return weekday_names.get(weekday_number, "无效的星期几")


data_0008291=data_000829.copy()
length=len(data_0008291.iloc[:,0])
for i in range(length):
    date=data_0008291.iloc[i,0]
    date_str=str(date)[0:4]+"-"+str(date)[4:6]+"-"+str(date)[6:]
    date_datetime = datetime.strptime(date_str, "%Y-%m-%d").date()
    weekday_number = date_datetime.weekday()  # Monday = 0, Sunday = 6
    data_0008291.at[i,'trade_date']=date_str+','+weekday_to_chinese(weekday_number)



In [15]:
#保存数据
data_0008291.to_excel('data/000829.xlsx', index=False)

Part 2:Data Perprocess

In [16]:
from deal import read_and_deal 
data=read_and_deal('data/000829.xlsx')
print(data)
print(data[1])

(array([   0,    1,    2, ..., 9157, 9158, 9159]), array([[11.82, 11.63, 12.  , 11.41],
       [11.5 , 10.7 , 11.5 , 10.65],
       [10.51, 11.15, 11.59, 10.48],
       ...,
       [10.06,  9.93, 10.12,  9.92],
       [ 9.86,  9.5 ,  9.89,  9.49],
       [ 9.66,  9.74,  9.76,  9.45]]))
[[11.82 11.63 12.   11.41]
 [11.5  10.7  11.5  10.65]
 [10.51 11.15 11.59 10.48]
 ...
 [10.06  9.93 10.12  9.92]
 [ 9.86  9.5   9.89  9.49]
 [ 9.66  9.74  9.76  9.45]]


In [19]:
#将数据储存在numpy的数组
np.savez('data/static.npz', t=data[0], data=data[1], allow_pickle=True)

Part3:Training

In [3]:
!python train.py --data data/static.npz --config Cfg.yaml

Making dataset...
Done!
CNN_LSTM-开盘 training...

训练进度(1/200轮):0.50%[->-------------------------------------------------] 用时:1.56s 验证集上损失:0.377 
训练进度(2/200轮):1.00%[->-------------------------------------------------] 用时:3.07s 验证集上损失:0.335 
训练进度(3/200轮):1.50%[->-------------------------------------------------] 用时:4.55s 验证集上损失:0.323 
训练进度(4/200轮):2.00%[▓->-------------------------------------------------] 用时:6.05s 验证集上损失:0.295 
训练进度(5/200轮):2.50%[▓->------------------------------------------------] 用时:7.55s 验证集上损失:0.305 
训练进度(6/200轮):3.00%[▓->------------------------------------------------] 用时:9.05s 验证集上损失:0.324 
训练进度(7/200轮):3.50%[▓->------------------------------------------------] 用时:10.90s 验证集上损失:0.304 
训练进度(8/200轮):4.00%[▓▓->------------------------------------------------] 用时:13.25s 验证集上损失:0.303 
训练进度(9/200轮):4.50%[▓▓->-----------------------------------------------] 用时:15.47s 验证集上损失:0.302 
训练进度(10/200轮):5.00%[▓▓->-----------------------------------------------] 用时:17.69s 验证集上损失:0

Part 4:Test

In [4]:
!python test.py --data data/static.npz --config Cfg.yaml

model: CNN_LSTM-开盘
time per prediction: 0.713 ms
MSE: 1.440967521109162
RMSE: 1.2004030661028662
MAE: 0.7082274402245095
R2: 0.9416765539936091
MAPE: 5.755916328941744 %
SMAPE: 5.665421111885735 %
